# Global averaging

In [23]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import xarray as xr
import intake
import os
import importlib
import my_functions as f
importlib.reload(f) # needed to load new changes in my_functions

col_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(col_url)

## Choose model, exp, member

In [24]:
#model = 'CNRM-CM6-1'
#model = 'EC-Earth3' # should have spatial dimensions 256, 512
#model = 'GISS-E2-1-G'
model = 'ACCESS-CM2'
#model = 'HadGEM3-GC31-LL'
#model = 'CMCC-CM2-HR4'
#model = 'FIO-ESM-2-0'
#model = 'KIOST-ESM'

print(model)

#check what experiments are available for var
var = 'tas'
#cat = col.search(experiment_id = ['piControl', 'abrupt-4xCO2', 'abrupt-2xCO2', 'abrupt-0p5xCO2', '1pctCO2', 'historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585'], source_id = model, variable_id=var, table_id='Amon') 
#cat = col.search(experiment_id = ['piControl', 'abrupt-4xCO2', 'abrupt-2xCO2', 'abrupt-0p5xCO2', '1pctCO2', 'historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585'], source_id = model, variable_id=var, table_id='Amon') 
#cat = col.search(experiment_id = 'piControl', source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon') 
#cat = col.search(experiment_id = 'ssp245', source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon') 
#cat = col.search(experiment_id = '1pctCO2', source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon') 
cat = col.search(experiment_id = 'abrupt-4xCO2', source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon') 
#cat = col.search(experiment_id = 'piClim-control', source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon') 
#cat = col.search(experiment_id = 'historical', source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon') 

#cat.df
pd.set_option('display.max_colwidth', -1)
cat.df['zstore']

ACCESS-CM2


0    gs://cmip6/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/abrupt-4xCO2/r1i1p1f1/Amon/rlut/gn/v20191108/
1    gs://cmip6/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/abrupt-4xCO2/r1i1p1f1/Amon/rsdt/gn/v20191108/
2    gs://cmip6/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/abrupt-4xCO2/r1i1p1f1/Amon/rsut/gn/v20191108/
3    gs://cmip6/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/abrupt-4xCO2/r1i1p1f1/Amon/tas/gn/v20191108/ 
Name: zstore, dtype: object

In [25]:
#exp = 'piControl'
exp = 'abrupt-4xCO2'
#exp = 'abrupt-2xCO2'
#exp = 'abrupt-0p5xCO2'
#exp = '1pctCO2'
#exp = 'historical'
#exp = 'ssp126'
#exp = 'ssp245'
#exp = 'ssp370'
#exp = 'ssp585'
#exp = 'piClim-4xCO2'
#exp = 'piClim-control'
#exp = 'piClim-histall'


if model == 'UKESM1-0-LL':
    cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', institution_id = 'MOHC')
    #cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', institution_id = 'NIMS-KMA')
if model == 'MPI-ESM1-2-HR': # select institution if there are two groups
    cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
    #cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', institution_id = 'DKRZ') 
    #cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', institution_id = 'DWD') 
elif model in ['MPI-ESM1-2-LR', 'IPSL-CM6A-LR', 'MRI-ESM2-0', 'EC-Earth3'] and exp in ['piControl']: # problem when loading more than one member simultaneously, so specify member_id:
    cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', member_id = 'r2i1p1f1')
else:
    cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
    #cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', member_id = 'r8i1p5f1')
    #                 member_id = ['r2i1p1f1','r3i1p1f1','r4i1p1f1','r5i1p1f1','r6i1p1f1','r7i1p1f1','r8i1p1f1','r9i1p1f1','r10i1p1f1','r11ip1f1','r12i1p1f1']) 
    #cat = col.search(experiment_id = exp, source_id = model, variable_id=['rlut'], table_id='Amon', member_id = 'r1i1p1f1')

# members with starting months different from jan should be loaded separately to ensure we get the correct time axis


if 'dset_dict' in locals():
    del dset_dict
if 'ds_exp' in locals():
    del ds_exp
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})
#for key in ['CMIP.MOHC.UKESM1-0-LL.historical.Amon.gn']:
for key in dset_dict.keys():
    ds_exp = dset_dict[key]
    if 'members_sorted' in locals():
        del members_sorted
    members_sorted = ds_exp.member_id.sortby(ds_exp.member_id)
    
if model == 'MCM-UA-1-0':
    ds_exp = ds_exp.rename({'longitude': 'lon','latitude': 'lat'}) 
        
for member in members_sorted:
    print(member.values)
    
# write out data variables, to check that we have all we want
ds_exp.data_vars


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


r1i1p1f1


Data variables:
    rlut     (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 540, 144, 192), meta=np.ndarray>
    rsdt     (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 900, 144, 192), meta=np.ndarray>
    rsut     (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 553, 144, 192), meta=np.ndarray>
    tas      (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 669, 144, 192), meta=np.ndarray>

In [26]:
ds_exp

<xarray.Dataset>
Dimensions:    (lat: 144, bnds: 2, lon: 192, member_id: 1, time: 1800)
Coordinates:
  * lat        (lat) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(144, 2), meta=np.ndarray>
  * lon        (lon) float64 0.9375 2.812 4.688 6.562 ... 355.3 357.2 359.1
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
  * time       (time) object 0950-01-16 12:00:00 ... 1099-12-16 12:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(1800, 2), meta=np.ndarray>
  * member_id  (member_id) <U8 'r1i1p1f1'
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    rlut       (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 540, 144, 192), meta=np.ndarray>
    rsdt       (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 900, 144, 192), meta=np.ndarray>
    rsut       (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 553, 144, 192), meta=np.ndarray>
    tas        (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 669, 144, 192), meta=np.ndarray>
Attributes: (12/48)
    institution_id:          CSIRO-ARCCSS
    activity_id:             CMIP
    realm:                   atmos
    version:                 v20191108
    table_info:              Creation Date:(30 April 2019) MD5:e14f55f257ccea...
    source:                  ACCESS-CM2 (2019): \naerosol: UKCA-GLOMAP-mode\n...
    ...                      ...
    run_variant:             forcing: GHG, Oz, SA, Sl, Vl, BC, OC, (GHG = CO2...
    nominal_resolution:      250 km
    version_id:              v20191108
    grid:                    native atmosphere N96 grid (144x192 latxlon)
    parent_mip_era:          CMIP6
    intake_esm_dataset_key:  CMIP.CSIRO-ARCCSS.ACCESS-CM2.abrupt-4xCO2.Amon.gn

In [27]:
#ds_exp.time.to_index()
ds_exp.time.encoding['calendar']

'proleptic_gregorian'

In [28]:
if model in ['CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CNRM-ESM2-1', 'IPSL-CM6A-LR']: # French models missing lat_bnds and lon_bnds coordinates
    # if computing area-weights from areacella instead:
    #area_cat= col.search(source_id = model, variable_id=['areacella'])
    area_cat = col.search(experiment_id = 'piControl', source_id = model, variable_id=['areacella'])
    area_dset_dict = area_cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})

    for key in area_dset_dict.keys():
        area_ds = area_dset_dict[key]
    areas = area_ds['areacella'].values[0,:,0]
    norm_areas = areas/areas.mean()

# loop through members
#for member in [members_sorted.sel(member_id = 'r1i1p1f3')]:
for member in members_sorted:
    print(member.values)

    ds = ds_exp.sel(member_id = member)
    
    
    ds_calendar = ds.time.encoding['calendar']
    #ds_calendar = 'proleptic_gregorian'
    unit = ds.time.encoding['units']
    firstmonth = ds_exp.time.to_index().month[0] # not jan for IPSL-CM6A-LR members 2-12
    # annual average from feb-jan, march-feb, etc. when firstmonth is not jan. 
    
    # anomalies: does it matter if the piControl annual averages we subtract are averaged over jan-dec or other months? Probably not
    if model == 'MRI-ESM2-0' and exp == 'abrupt-4xCO2':
        # note: piControl values in these time series before the branch time
        rstring = str(member.values)[1:3].replace('i','')
        firstmonth = int(rstring)
        if firstmonth>1: # delete values before 'firstmonth', and some values in the end:
            #ds = ds.drop_isel(time=np.arange(firstmonth-1))
            ds = ds.isel(time=np.arange(firstmonth-1,len(ds.time)-(12-firstmonth+1)))
            # check result
            print(ds.time.to_index().month[0])
    elif model == 'CNRM-CM6-1' and exp == 'abrupt-4xCO2':
        # delete values in the end to get a multiple of 12 length for the annual mean computations
        ds = ds.isel(time=np.arange(len(ds.time)-(12-firstmonth+1)))
        
    
    print(ds_calendar, 'calendar')
    print(unit)
    print('first month of dataset is:', firstmonth)
    
    # compute weights for average
    if model == 'NorCPM1' and exp == 'historical':
        area_w = f.area_weights(ds.lat_bnds.values[0,:,:], ds.lon_bnds.values)
    elif model in ['CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CNRM-ESM2-1', 'IPSL-CM6A-LR']: # French models missing lat_bnds and lon_bnds coordinates
        area_w = norm_areas
        #print('maximum difference in area-weights is:', np.max(np.abs(norm_areas - area_w)))
        
        # differences in global means are noticeable from the third decimal for the low-res model BCC-ESM1
        # for this model maximum difference in area-weights is: 0.003299741144367374
        
        # differences in global means are noticeable from the 5th decimal for the model CESM2
        # for this model maximum difference in area-weights is: 1.2547306571519812e-07
    else:
        area_w = f.area_weights(ds.lat_bnds.values, ds.lon_bnds.values)
        
    day_weights = f.compute_day_weights(ds, calendar = ds_calendar, first_month = firstmonth)

    varlist = ['tas', 'rlut', 'rsut', 'rsdt']
    
    for variable in varlist:
        print(variable)
        data = ds[variable]
                
        # global average
        area_avg = (data.transpose('time', 'lon', 'lat') * area_w).mean(dim=['lon', 'lat'])

        # annual average
        day_weighted_avg = area_avg*day_weights
        if firstmonth == 1 or firstmonth == 13:
            annualmean = day_weighted_avg.groupby('time.year').mean('time')
        else: # this code is slow, but rarely needed
            annualmean_array = np.array([day_weighted_avg[i*12:(i+1)*12].mean() for i in range(int(len(day_weighted_avg)/12))])
            annualmean = xr.DataArray(annualmean_array)
            
        if variable == varlist[0]:
            # create dataframe for storing all results
            df = pd.DataFrame(annualmean.values, columns = [variable])
        else:
            df_col = pd.DataFrame(annualmean.values, columns = [variable])
            df = pd.merge(df, df_col, left_index=True, right_index=True, how='outer')
            
    filename = model + '_' + exp + '_' + str(member.values) + '_means.txt'
    filepath = os.path.join('../Processed_data/Global_annual_means/', model, exp)
    if os.path.isdir(filepath) == False:
        os.makedirs(filepath)
    else:
        if member == members_sorted[0]:
            print('path already exists. Make sure no data in it are written over erroneously')

    #df.to_csv(filepath + '/' + filename)
    

r1i1p1f1
proleptic_gregorian calendar
hours since 0950-01-16 12:00:00.000000
first month of dataset is: 1
tas
rlut
rsut
rsdt
path already exists. Make sure no data in it are written over erroneously


In [29]:
ds.time


<xarray.DataArray 'time' (time: 1800)>
array([cftime.DatetimeProlepticGregorian(950, 1, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeProlepticGregorian(950, 2, 15, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeProlepticGregorian(950, 3, 16, 12, 0, 0, 0, has_year_zero=True),
       ...,
       cftime.DatetimeProlepticGregorian(1099, 10, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeProlepticGregorian(1099, 11, 16, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeProlepticGregorian(1099, 12, 16, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time       (time) object 0950-01-16 12:00:00 ... 1099-12-16 12:00:00
    member_id  <U8 'r1i1p1f1'
    height     float64 ...
Attributes:
    axis:           T
    bounds:         time_bnds
    long_name:      time
    standard_name:  time

In [30]:
df

,tas,rlut,rsut,rsdt
0,288.092920,237.856152,94.697509,340.318251
1,289.032851,240.380828,93.538969,340.318251
2,289.337434,241.218091,93.453865,340.349984
3,289.764315,242.464048,93.074863,340.318251
4,289.832869,242.310315,92.522738,340.318251
...,...,...,...,...
145,293.821067,251.423404,86.331730,340.318251
146,293.715534,251.209791,86.719511,340.349984
147,293.612279,251.135229,86.705655,340.318251
148,293.845368,251.688767,86.338357,340.318251
